# HSS LM Train

### Load

In [7]:
import pandas as pd
from tqdm.auto import tqdm
from kiwipiepy import Kiwi
kiwi = Kiwi()
from mpire import WorkerPool
import os
from itertools import chain
import pickle

In [ ]:
# hu = pd.read_excel('./data/인문학분야 KCI 논문 정보 추출_20240216.xlsx')
# ss = pd.read_excel('./data/사회과학분야 KCI 논문 정보추출_20240216.xlsx')

In [2]:
df = pd.read_pickle('./data/KCI_HSS.pkl')
df

,논문ID,주저자명,공동저자명,주저자소속기관,주저자ORCID,논문제목,논문외국어제목,논문영어제목,학술지ID,학술지명,...,키워드,외국어키워드,영어키워드,언어,참고문헌수,피인용횟수,DOI,주제분야,URL,등재구분
0,ART001134609,강인욱,NaN,서울대학교,NaN,韓半島 出土 動物形 帶鉤의 系統에 대한 시론적 검토,NaN,NaN,000669,호남고고학보,...,"동물형 대구, 마형대구, 호형대구, 동물장식, 세형동검문화",NaN,NaN,NaN,59,16,NaN,인문학 > 역사학,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재
1,ART001134565,안승모,이영덕,원광대학교,NaN,龍潭댐 水沒地區의 新石器文化-鎭安 갈머리遺蹟을 中心으로-,NaN,NaN,000669,호남고고학보,...,"신석기시대, 즐문토기, 금강, 용담댐, 갈머리유적",NaN,NaN,NaN,39,5,NaN,인문학 > 역사학,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재
2,ART001134567,노혁진,NaN,한림대학교,NaN,中島式吐器의 由來에 대한 一考,NaN,NaN,000669,호남고고학보,...,"중도식토기, 끄로우노프까문화, 호곡동, 강원영동, 북한강권, 동북전파가설",NaN,NaN,NaN,33,29,NaN,인문학 > 역사학,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재
3,ART001134520,이기길,NaN,조선대학교,NaN,진안 진그늘유적 구석기문화층의 성격과 의미,NaN,NaN,000669,호남고고학보,...,"진그늘유적, 유적 구조, 유적 기능, 석기 무리, 화덕자리, 슴베찌르개, 돌날제작...",NaN,NaN,NaN,23,27,NaN,인문학 > 역사학,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재
4,ART001134566,김승옥,NaN,전북대학교,NaN,龍潭댐 無文土器時代 文化의 社會組織과 變遷過程,NaN,NaN,000669,호남고고학보,...,"용담댐 무문토기사회, 사회조직과 변천, 가족 공동체, 유력 가족묘, 유력 개인묘,...",NaN,NaN,NaN,35,11,NaN,인문학 > 역사학,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834710,ART003038503,류수연,박성남|최가윤,건축공간연구원,NaN,인구감소지역의 사람과 장소 측면의 현안 및 공간전략,The Current Issues And Spatial Strategies Conc...,The Current Issues And Spatial Strategies Conc...,001620,도시설계,...,"인구감소지역, 지방소멸대응기금, 사람 및 장소 현안, 공간전략","Depopulated Area, Local Extinction Response Fu...","Depopulated Area, Local Extinction Response Fu...",한국어,0,0,NaN,사회과학 > 국제/지역개발 > 지역개발 > 도시개발/계획,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재
834711,ART003038530,정상규,김난영|반영운,충북대학교,NaN,자원순환형 도시정원 조성을 위한 의사결정 요인,Decision-making Factors for Urban Gardening to...,Decision-making Factors for Urban Gardening to...,001620,도시설계,...,"도시정원, 자원순환, 의사결정 요인, 순환경제, 동시출현 네트워크","Urban Garden, Resource Cycling, Decision-makin...","Urban Garden, Resource Cycling, Decision-makin...",한국어,0,0,NaN,사회과학 > 국제/지역개발 > 지역개발 > 도시개발/계획,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재
834712,ART003038545,남예인,이수기|이은영|김유미,한양대학교,NaN,거주민의 우울에 영향을 미치는 근린환경 특성 분석 : 2019년 경기도 지역사회건강...,Analysis of Neighborhood Environments Affectin...,Analysis of Neighborhood Environments Affectin...,001620,도시설계,...,"우울, 근린환경, 거리뷰 이미지, 의미론적 분할 기법, 지역사회건강조사","Depression, Neighborhood Environment, Street V...","Depression, Neighborhood Environment, Street V...",한국어,0,0,NaN,사회과학 > 국제/지역개발 > 지역개발 > 도시개발/계획,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재
834713,ART003038514,권익현,정도훈|유석연,서울시립대학교 도시과학연구원,NaN,소규모주택정비 관리지역의 유형별 특성 - 서울시 모아타운 1차 선정지역을 중심으로,Characteristics of Small-scale Housing Improve...,Characteristics of Small-scale Housing Improve...,001620,도시설계,...,"소규모주택정비 관리지역, 모아타운, 지역특성, 유형",Small-scale Housing Improvement Management Are...,Small-scale Housing Improvement Management Are...,한국어,0,0,NaN,사회과학 > 국제/지역개발 > 지역개발 > 도시개발/계획,https://www.kci.go.kr/kciportal/ci/sereArticle...,등재


In [44]:
# df = pd.concat([hu,ss],ignore_index=True)
# df = hu
# del(hu,ss)

In [6]:
df.columns

Index(['논문ID', '주저자명', '공동저자명', '주저자소속기관', '주저자ORCID', '논문제목', '논문외국어제목',
       '논문영어제목', '학술지ID', '학술지명', '학술지약어명', '학술지외국어명', '발행기관ID', '발행기관명',
       '발행기관영어명', 'ISSN', 'EISSN', '발행년도', '발행월/일', '권', '호', '특별호', '시작페이지',
       '끝페이지', '초록', '영어초록', '키워드', '외국어키워드', '영어키워드', '언어', '참고문헌수', '피인용횟수',
       'DOI', '주제분야', 'URL', '등재구분'],
      dtype='object')

In [8]:
# df.to_pickle('./data/KCI_HSS.pkl')

In [3]:
# titles_abstracts = df.dropna(subset=['논문제목'])['논문제목'].tolist() + df.dropna(subset=['초록'])['초록'].tolist()
titles = df.dropna(subset=['논문제목'])['논문제목'].tolist()

In [4]:
# split sents
def split_into_sents(text):
    res = kiwi.split_into_sents(text)
    return [t.text for t in res]

In [5]:
n_cpu = os.cpu_count()
with WorkerPool(n_jobs=n_cpu-1) as pool:
    abs_split = pool.map(split_into_sents, df.dropna(subset=['초록'])['초록'].tolist(), progress_bar=True)

100%|██████████| 769540/769540 [05:41<00:00, 560.11it/s] 


In [6]:
abs_split_list = list(chain(*abs_split))
len(abs_split_list)

5523767

In [11]:
with open('./data/abs_split_list.pkl','wb') as f:
    pickle.dump(abs_split_list,f)

### Train

In [13]:
from torch.utils.data import Dataset
import torch

class ListDataset(Dataset):
    def __init__(self, tokenizer, texts, max_length):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        return {key: torch.squeeze(value) for key, value in encoding.items()}

In [16]:
# output_dir='./TitleEpoch3'
output_dir='./HssTitleAbsEpoch10'
num_train_epochs = 10
texts = titles + abs_split_list

In [15]:
len(texts)

6358481

In [17]:
from transformers import RobertaConfig, RobertaForMaskedLM, RobertaTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# 토크나이저 및 모델 초기화
tokenizer = RobertaTokenizerFast.from_pretrained('klue/roberta-large')
model = RobertaForMaskedLM(config=RobertaConfig())

# 데이터셋 준비
# texts = ["여기에 한국어 텍스트 데이터를 리스트 형태로 넣어주세요.", "데이터 사이언스는 재미있습니다.", "파이토치는 딥러닝 프레임워크입니다."]
dataset = ListDataset(tokenizer, texts, max_length=128)

# 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# 트레이닝 아규먼트
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# 트레이너 초기화 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

{'loss': 1.2992, 'learning_rate': 4.411106022556278e-07, 'epoch': 9.91}
{'loss': 1.3141, 'learning_rate': 4.285290106339612e-07, 'epoch': 9.91}


/home/bjkim/miniconda3/envs/py311_tf/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.3146, 'learning_rate': 4.1594741901229476e-07, 'epoch': 9.92}
{'loss': 1.3021, 'learning_rate': 4.0336582739062824e-07, 'epoch': 9.92}
{'loss': 1.2835, 'learning_rate': 3.907842357689618e-07, 'epoch': 9.92}
{'loss': 1.3086, 'learning_rate': 3.782026441472952e-07, 'epoch': 9.92}
{'loss': 1.3052, 'learning_rate': 3.656210525256287e-07, 'epoch': 9.93}
{'loss': 1.3153, 'learning_rate': 3.530394609039622e-07, 'epoch': 9.93}
{'loss': 1.3078, 'learning_rate': 3.4045786928229566e-07, 'epoch': 9.93}
{'loss': 1.3055, 'learning_rate': 3.2787627766062915e-07, 'epoch': 9.93}
{'loss': 1.3065, 'learning_rate': 3.152946860389627e-07, 'epoch': 9.94}
{'loss': 1.3109, 'learning_rate': 3.0271309441729617e-07, 'epoch': 9.94}
{'loss': 1.32, 'learning_rate': 2.901315027956297e-07, 'epoch': 9.94}
{'loss': 1.3105, 'learning_rate': 2.775499111739632e-07, 'epoch': 9.94}
{'loss': 1.2874, 'learning_rate': 2.649683195522967e-07, 'epoch': 9.95}
{'loss': 1.3008, 'learning_rate': 2.5238672793063016e-07, 'ep

/home/bjkim/miniconda3/envs/py311_tf/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.3052, 'learning_rate': 1.6431558657896458e-07, 'epoch': 9.97}
{'loss': 1.3206, 'learning_rate': 1.517339949572981e-07, 'epoch': 9.97}
{'loss': 1.3019, 'learning_rate': 1.3915240333563157e-07, 'epoch': 9.97}
{'loss': 1.3024, 'learning_rate': 1.2657081171396506e-07, 'epoch': 9.97}
{'loss': 1.3075, 'learning_rate': 1.1398922009229855e-07, 'epoch': 9.98}
{'loss': 1.2927, 'learning_rate': 1.0140762847063205e-07, 'epoch': 9.98}
{'loss': 1.3149, 'learning_rate': 8.882603684896555e-08, 'epoch': 9.98}
{'loss': 1.3185, 'learning_rate': 7.624444522729905e-08, 'epoch': 9.98}
{'loss': 1.3023, 'learning_rate': 6.366285360563253e-08, 'epoch': 9.99}
{'loss': 1.294, 'learning_rate': 5.108126198396602e-08, 'epoch': 9.99}
{'loss': 1.3033, 'learning_rate': 3.849967036229951e-08, 'epoch': 9.99}
{'loss': 1.3134, 'learning_rate': 2.5918078740633005e-08, 'epoch': 9.99}
{'loss': 1.3083, 'learning_rate': 1.3336487118966498e-08, 'epoch': 10.0}
{'loss': 1.2973, 'learning_rate': 7.548954972999905e-10, '

TrainOutput(global_step=1987030, training_loss=1.6313008470043282, metrics={'train_runtime': 529260.6436, 'train_samples_per_second': 120.139, 'train_steps_per_second': 3.754, 'train_loss': 1.6313008470043282, 'epoch': 10.0})

In [18]:
# 모델 저장하기
model.save_pretrained(output_dir)
# 토크나이저 저장하기
tokenizer.save_pretrained(output_dir)

('./HssTitleAbsEpoch10/tokenizer_config.json',
 './HssTitleAbsEpoch10/special_tokens_map.json',
 './HssTitleAbsEpoch10/vocab.txt',
 './HssTitleAbsEpoch10/added_tokens.json',
 './HssTitleAbsEpoch10/tokenizer.json')

### Finetune

In [7]:
from transformers import RobertaTokenizerFast, RobertaConfig, RobertaForMaskedLM, AdamW, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader, Dataset
import torch

class KoreanTextDataset(Dataset):
    def __init__(self, tokenizer, data, block_size=128):
        self.examples = tokenizer(data, add_special_tokens=True, truncation=True, max_length=block_size, padding="max_length", return_tensors="pt")

    def __len__(self):
        return len(self.examples['input_ids'])

    def __getitem__(self, i):
        # Dataset은 input_ids와 attention_mask를 모두 반환합니다.
        return {'input_ids': self.examples['input_ids'][i], 'attention_mask': self.examples['attention_mask'][i]}

def train(model, dataset, device, batch_size=16, epochs=1):
    model = torch.nn.DataParallel(model).to(device)

    # Data collator를 사용하여 자동으로 패딩을 적용합니다.
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)

    optimizer = AdamW(model.parameters(), lr=5e-5)

    model.train()
    for epoch in range(epochs):
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()} # 배치 내 모든 텐서를 디바이스로 이동
            outputs = model(**batch)
            # loss = outputs.loss
            loss = outputs.loss.mean()  # 여러 GPU에서 반환된 손실 값의 평균을 계산
            # if not torch.is_tensor(loss) or not loss.dim() == 0:
            #     raise ValueError(f"Expected scalar tensor, got: {loss}")
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(f"Epoch: {epoch}, Loss: {loss.item()}")
            
if __name__ == "__main__":
    # tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
    tokenizer = RobertaTokenizerFast.from_pretrained('klue/roberta-large')
    # model = RobertaForMaskedLM.from_pretrained('roberta-base')
    model = RobertaForMaskedLM.from_pretrained("klue/roberta-large")

    # 데이터 리스트 예시
    # data = ["여기에 한국어 텍스트 데이터를 리스트 형태로 넣어주세요.", "데이터 사이언스는 재미있습니다.", "파이토치는 딥러닝 프레임워크입니다."]

    dataset = KoreanTextDataset(tokenizer, texts)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train(model, dataset, device)